In [ ]:
%matplotlib notebook 

In [ ]:
import itertools
from IPython.display import Audio
from matplotlib import animation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 6

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

# Partial differential equations

## Solving Laplace's equation

Consider Laplace's equation in the form $V_{xx} + V_{yy} = 0$, where $V(x, y)$ is a potential. This is something we want to solve numerically. We can do so by using a grid with step size $h$.

On this grid, use the notation $V(x_i, y_j) = V_{i, j}$. A finite difference formula for the second derivative is

\begin{equation}
    V_{xx} = \frac{V_{i - 1, j} - 2 V_{i, j} + V_{i + 1, j}}{h^2},
\end{equation}

and similarly for $V_{yy}$.

We can come up with a nice equation for the potential at a grid point $(i, j)$ as follows:

\begin{align}
    &V_{xx} + V_{yy} = 0 \\
    &\Rightarrow 
        \frac{V_{i - 1, j} - 2 V_{i, j} + V_{i + 1, j}}{h^2}
        + \frac{V_{i, j - 1} - 2 V_{i, j} + V_{i, j + 1}}{h^2}
        = 0 \\
    &\Rightarrow V_{i, j} = \frac{1}{4} \left(
        V_{i - 1, j} + V_{i + 1, j} + V_{i, j - 1} + V_{i, j + 1}
        \right)
    .
\end{align}

See the discussion in the lab script for more details on numerical methods for solving Laplace's equation.

### Potential with fixed boundaries

Consider Laplace's equation in a square box, 1m on each side, with voltage 1V along the bottom and far wall and 0V along the others.

In [ ]:
# Set up the box matrix
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[:, -1] = np.ones(101)
box_mat[-1, :] = np.ones(101)

# Tolerance
delta_tol = 1e-4

#### Jacobi update formula

Here we will solve for the potential numerically using the Jacobi update formula.

In [ ]:
iters = 0
max_diff_sqrd = 1

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Jacobi update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        box_mat[i, j] = (
            old_mat[i - 1, j]
            + old_mat[i + 1, j]
            + old_mat[i, j - 1]
            + old_mat[i, j + 1]
        ) / 4

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

This took the following number of iterations:

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

#### Gauss-Seidel method

In [ ]:
# Set up the box matrix
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[:, -1] = np.ones(101)
box_mat[-1, :] = np.ones(101)

# Tolerance
delta_tol = 1e-4

Now we'll try using the Gauss-Seidel method using the same tolerance as before.

In [ ]:
iters = 0
max_diff_sqrd = 1

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Gauss-Seidel update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        box_mat[i, j] = (
            box_mat[i - 1, j]
            + box_mat[i + 1, j]
            + box_mat[i, j - 1]
            + box_mat[i, j + 1]
        ) / 4

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

For this method, the Gauss-Seidel used 74% of the iterations used by the Jacobi solution, and appears to give a similar solution.

#### Gauss-Seidel with SOR method

Finally, let's try the SOR method using $\omega = 1.2$.

In [ ]:
# Set up the box matrix
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[:, -1] = np.ones(101)
box_mat[-1, :] = np.ones(101)

# Tolerance
delta_tol = 1e-4

In [ ]:
iters = 0
max_diff_sqrd = 1

# SOR weight
weight = 1.2

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Gauss-Seidel update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        box_mat[i, j] = (
            box_mat[i - 1, j]
            + box_mat[i + 1, j]
            + box_mat[i, j - 1]
            + box_mat[i, j + 1]
        ) / 4

    # Apply SOR method
    box_mat = (1 - weight) * old_mat + weight * box_mat

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

This method gave us a slight speed up for this example. Nothing dramatic.

### Potential of a parallel plate capacitor

Now we will solve the parallel plate capacitor problem in the lab script using the Gauss-Seidel method.

In [ ]:
# Set up the box matrix. The initial guess
# will be zeroes everywhere we do not have
# a fixed value.
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[20:80, 30] = np.ones(60)
box_mat[20:80, 70] = - np.ones(60)

# Tolerance
delta_tol = 1e-4

Now we'll try using the Gauss-Seidel method using the same tolerance as before.

In [ ]:
iters = 0
max_diff_sqrd = 1

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Gauss-Seidel update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        # Skip over the capacitor "plates"
        if 20 <= i <= 79 and j in (30, 70):
            continue

        box_mat[i, j] = (
            box_mat[i - 1, j]
            + box_mat[i + 1, j]
            + box_mat[i, j - 1]
            + box_mat[i, j + 1]
        ) / 4

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

## Solving the heat/diffusion equation

Similarly to Laplace's equation we can discretize the heat equation to derive the FTCS equation, which provides a numerical method that we can use to solve the heat equation.

### Problems

#### Thermal diffusion through Earth

First, using the formulas provided in the lab script, we will plot the seasonal variation of the ground temperature in Canada's arctic.

In [ ]:
# Constants
A = -12  # deg C
B = 20  # deg C
phi = -1.9  # rads

# Get data
days = np.arange(1, 366, 1)
ground_Ts = A + B * np.sin(2 * np.pi * days / 365 + phi)

In [ ]:
# Setup figure
_, ax = plt.subplots()

plt.plot(days, ground_Ts)

# Labels
ax.set_xlabel(r"$t$ (days)")
ax.set_ylabel(r"ground $T$ ($^\circ C$)");

Now let's consider the heat equation on the domain $[0, \infty) \times [0, L]$ (where the first term in the Cartesian product is the temporal part and the second term is the spacial part), with boundary conditions

\begin{align}
    T(0, x) &= A, \\
    T(t, 0) &= A + B \sin \left( \frac{2 \pi t}{\tau} + \phi \right), \\
    T(t, L) &= A + m_T L.
\end{align}

We will solve the heat equation over 10 years numerically.

In [ ]:
# Add in extra constants
alpha = 0.1  # m^2 / day
m_T = 30e-3  # deg C / m
L = 40  # m

num_years = 10
num_spatial_pts = 101

h = L / (num_spatial_pts - 1)
approx_dt = 0.7

num_ts_per_year = round(365 / approx_dt)

dt = 365 / num_ts_per_year

In [ ]:
print(dt < (h ** 2 / (2 * alpha)))

Now that we've verified that our solution will be stable, we can go ahead and get solve numerically.

In [ ]:
ts = np.tile(np.arange(0, 365, dt), num_years)
num_ts = ts.shape[0]

In [ ]:
# Set up data matrix
data_mat = A * np.zeros((num_spatial_pts, num_ts))

# Enforce boundary conditions
data_mat[0, :] = A + B * np.sin(2 * np.pi * ts / 365 + phi)
data_mat[-1, :] = A + m_T * L

In [ ]:
# Multiplicative constant in iteration formula
c = alpha * dt / h ** 2

for k in range(1, num_ts):
    for i in range(1, num_spatial_pts - 1):
        data_mat[i, k] = data_mat[i, k - 1] + c * (
            data_mat[i - 1, k - 1] - 2 * data_mat[i, k - 1] + data_mat[i + 1, k - 1]
        )

For now, we have way more data than we need. So let's grab 12 equidistant profiles from the last year we computed.

In [ ]:
# Num data points per month
pts_per_month = round(num_ts / num_years / 12)

target_data_idxs = num_ts + np.arange(-11, 1) * pts_per_month - 1

last_year_profiles = data_mat[:, target_data_idxs]

Now we'll plot four of the profiles separated by three month intervals.

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(8, 12), sharey=True)

plt.subplots_adjust(hspace=0.7)

xs = np.arange(0, L + h, h)

for idx, ax in enumerate(axs):
    ax.plot(xs, last_year_profiles[:, 3 * idx])

    ax.set_xlabel(r"depth (m)")
    ax.set_ylabel(r"temperature ($^\circ C$)")

    ax.set_title("Month %d" % (3 * idx + 1))

Here we see that all significant fluctuations in temperature occur near the surface. At around a depth of 10m or more the temperature is more or less constant.

Now, using the 12 profiles we picked out, let's calculate the maximum and minimum yearly temperature as a function of depth.

In [ ]:
min_temps = last_year_profiles.min(axis=1)
max_temps = last_year_profiles.max(axis=1)

In [ ]:
_, ax = plt.subplots()

plt.plot(xs, max_temps)
plt.plot(xs, min_temps)

plt.grid(alpha=0.3)

ax.legend(["max yearly T", "min yearly T"])

ax.set_xlabel(r"depth (m)")
ax.set_ylabel(r"temperature ($^\circ C$)");

We can see here that permafrost occurs at a depth of around 2m.

## Solving the wave equation

For the wave equation we have an update formula similar to the heat equation.

### Problems

#### Wave on a string

Now we will simulate a guitar string.

In [ ]:
# Constants
L = 0.65  # m
c = 254.8  # m / s
f = 196  # Hz

A = 0.05  # m
d = 0.5  # m
sigma = 0.3  # m

sample_rate = 10 ** 4  # 1 / s
time_interval = 5  # s
num_spatial_pts = 51

dt = 1e-5  # s

num_ts = round(time_interval / dt)

dx = L / (num_spatial_pts - 1)

First we will solve the problem numerically.

In [ ]:
# Set up data matrix
data_mat = np.zeros((num_spatial_pts, num_ts))

# Set up initial shape
xs = np.arange(0, L + dx, dx)

data_mat[:, 0] = (
    A * xs * (L - xs) / L ** 2 * np.exp(-((xs - d) ** 2) / (2 * sigma ** 2))
)
data_mat[:, 1] = data_mat[:, 0]

In [ ]:
# Multiplicative constant in iteration formula
c1 = c ** 2 * dt ** 2 / dx ** 2

for k in range(2, num_ts):
    for i in range(1, num_spatial_pts - 1):
        data_mat[i, k] = (
            2 * data_mat[i, k - 1]
            - data_mat[i, k - 2]
            + c1
            * (data_mat[i - 1, k - 1] - 2 * data_mat[i, k - 1] + data_mat[i + 1, k - 1])
        )

We have too much data here. We only need as many samples as our sample rate.

In [ ]:
col_mult_to_keep = int(1 / (dt * sample_rate))

all_data_mat = data_mat
data_mat = np.copy(all_data_mat[:, ::col_mult_to_keep])

Now let's generate an animation of the waveform.

In [ ]:
# Get figure
fig, ax = plt.subplots()

# Limits
ax.set_xlim(0, L)
ax.set_ylim(-0.05, 0.05)

# Labels
ax.set_xlabel(r"$x$ (m)")

line = ax.plot(xs, data_mat[:, 0])[0]

# Update function for animation
def animate(i: int):
    line.set_data(xs, data_mat[:, i])
    return (line,)


ani = animation.FuncAnimation(fig, animate, frames=100, interval=20)

This animation runs really slowly for me. Not sure how to improve performance.

Let's test audio.

In [ ]:
ts = np.linspace(0, time_interval, sample_rate * time_interval)
data = np.sin(2 * np.pi * f * ts)

Audio(data, rate=sample_rate)

Now let's use our own data near the middle of the string.

In [ ]:
data = data_mat[num_spatial_pts // 2, :]

Audio(data, rate=sample_rate)

The audio sounds the same the test audio. Not identical, as the sound from our data has a bit higher pitch.

Now let's try near the end of the string.

In [ ]:
data = data_mat[num_spatial_pts - 2, :]

Audio(data, rate=sample_rate)

The note is the same, but the pitch is much higher.

Now we'll do a bit of FFT analysis.

In [ ]:
fs = np.fft.fftfreq(data_mat.shape[1], 1 / sample_rate)

We'll plot the spectrum for the first and second audio from our data

In [ ]:
_, (ax1, ax2) = plt.subplots(2, 1, sharey=True)

ak1 = np.fft.fft(data_mat[num_spatial_pts // 2, :])
ax1.plot(fs, np.abs(ak1 ** 2))

ak2 = np.fft.fft(data_mat[num_spatial_pts - 2, :])
ax2.plot(fs, np.abs(ak2 ** 2))

plt.yscale("log")